In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 

import utils.tavr_torch as tavr_torch
from utils.tavr_torch import TAVR_3_Frame, TAVR_1_Frame, TAVR_Sequence, tavr_dataloader
from utils.visualization import display_grid, z_stretch, visualize_frame, set_figsize, get_central_slices
from utils.loss_functions import batch_l2_loss
from utils.run_model import train, test, save, load, get_loss_history
from Models.basic_models import average_model, post_process
from Models.two_layer_resnet import two_layer_resnet

set_figsize(20, 15)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
USE_GPU = True
dtype = torch.float32 # we will be using float throughout this tutorial
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)

# "Pixl", "Slice", or "None"
preproc_type = "pixel"

validation = TAVR_3_Frame("__valid", preproc=preproc_type)
val_loader = tavr_dataloader(validation, batch_size=4, shuffle=True, num_workers=2)
training = TAVR_3_Frame("__train", preproc=preproc_type)
train_loader = tavr_dataloader(training,batch_size=8, shuffle=True, num_workers=2)


ave_model = average_model()
model = two_layer_resnet()
post_proc = post_process(kind=preproc_type).to(device=device)
loss_fn = batch_l2_loss()

# CHANGE TO NAME OF JUPYTER NOTEBOOK
model_name = "Model 7 (two_layer_resnet) Run 0"

using device: cpu


In [4]:
learning_rate = 3e-3
reg = 1e-7

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
# Run cell to load model
LOAD = False
iteration_num = -1

if LOAD:
    
    load(model_name, iteration_num, model, optimizer)
    loss_history = get_loss_history(model_name)
    model.to(device=device)
else:
    loss_history = None

In [6]:
train(model, post_proc, optimizer, train_loader, val_loader, loss_fn, device, 
         model_name, loss_history, epochs=3, print_every=30, print_level=4, lr_decay=1)


Iteration 0, loss = 1.4427, corrected loss = 671.3307
Validation loss 562.7200 over 81 frames
model saved to model_checkpoints/Model 7 (two_layer_resnet) Run 0/Model 7 (two_layer_resnet) Run 0-0
conv_a1.weight,   	norm: 3.8784e+00, 	update norm: 4.4088e-02 	Update/norm: 1.1368e-02
conv_a1.bias,   	norm: 1.9853e-01, 	update norm: 8.4852e-03 	Update/norm: 4.2740e-02
conv_b1.weight,   	norm: 4.2831e+00, 	update norm: 4.4091e-02 	Update/norm: 1.0294e-02
conv_b1.bias,   	norm: 2.5258e-01, 	update norm: 8.4852e-03 	Update/norm: 3.3594e-02
final.weight,   	norm: 1.3948e+00, 	update norm: 1.2000e-02 	Update/norm: 8.6035e-03
final.bias,   	norm: 1.4754e-01, 	update norm: 3.0000e-03 	Update/norm: 2.0334e-02

... 1.0319... 0.6162... 0.6140
Iter 10... 0.5726... 0.5383... 0.4951
Iter 20... 0.4515... 0.4118... 0.4205
Iteration 30, loss = 0.3845, corrected loss = 178.9088
Validation loss 161.6488 over 81 frames
model saved to model_checkpoints/Model 7 (two_layer_resnet) Run 0/Model 7 (two_layer_resn

In [ ]:
# One last test + visualize results on 1 validation sequence
val_seq = TAVR_Sequence("__valid")
test(model, val_loader, loss_fn, device)
test(ave_model, val_loader, loss_fn, device)

with torch.no_grad():
    model.eval()
    ave_frames = [post_proc(val_seq[0][0])]
    for i in range(2,9,2):
        ave_frame = model((val_seq[0][i-2][None,:], val_seq[0][i][None,:]))
        ave_frames += [post_proc(ave_frame[0][0]),  val_seq[0][i]]
    ave_frames += [val_seq[0][9]]
    ave_frames_slices = []
    for f in ave_frames:
        ave_frames_slices += get_central_slices(f)
    set_figsize(6,20)
    display_grid(10, 3, ave_frames_slices)

In [ ]:
optimizer.param_groups[0]['lr']

In [ ]:
for p in model.parameters():
    print(p.norm())